# What's new in Lightkurve v2.0?

## Overview

### Making Lightkurve more generic

Lightkurve v1.0 has been a tremendous success!  Owing to the efforts of 67 contributors, Lightkurve grew into a popular tool to analyze the data archive of NASA's Kepler space telescope.  To date, the toolkit has been cited or mentioned by nearly [200 scientific publications](https://ui.adsabs.harvard.edu/search/q=full%3A"Lightkurve").

In recent years, the success revealed a shortcoming: Lightkurve v1.0 was too narrowly focused on supporting official Kepler data products alone. Over the past two years, the Kepler user community has evolved in a number of important ways:

1. NASA successfully launched the TESS mission, adding a significant new data set which requires Lightkurve-like tools.
2. The data archive at MAST started hosting numerous community-created light curve databases extracted from Kepler and TESS.
3. Several new time series photometry missions are on the horizon, including ESA's Plato and NASA's Roman Space Telescope.

In light of these evolutions, there is a need for Lightkurve's tools to become more generically applicable to different data sets.


### `LightCurve` is now a sub-class of AstroPy's `TimeSeries` object

Recently, the AstroPy project added a fantastic new [astropy.timeseries](https://docs.astropy.org/en/stable/timeseries/) sub-package to support the analysis of generic time series data sets.
A decision has been made to transform Lightkurve's `LightCurve` object into a sub-class of AstroPy's `TimeSeries` object. This allows Lightkurve to build upon this generic, table-like data structure provided by AstroPy and focus more strongly on providing features that are specific to time resolved CCD photometry.

Three major benefits of this change are:

1. `LightCurve` objects can now contain arbitrary data columns (e.g. fluxes obtained using different methods).
2. Multiple light curve data formats are now supported via the generic `LightCurve.read()` and `LightCurve.write()` methods, which extend AstroPy's `Table.read()` and `Table.write()` functions.
3. `LightCurve` objects now leverage AstroPy `Time` objects to store the `time` column and AstroPy `Quantity` objects for all other columns. This enables us to support different time formats and data units in a generic way.



### How does Lightkurve's `LightCurve` differ from AstroPy's `TimeSeries`?

A `TimeSeries` object is a table which is guaranteed to have a special `time` column, which is always the first column and guaranteed to be an AstroPy `Time` object. `LightCurve` extends this concept by also adding special `flux` and `flux_err` columns, which are always the second and third column.  These dedicated columns enable us to provide extra methods which are specific to the manipulation of brightness data obtained by imaging photometry.  Some of these methods may eventually be migrated to `TimeSeries` if they are sufficiently generic and mature, but many methods that are specific Kepler/TESS-like data sets will likely remain in `LightCurve` to avoid overwhelming AstroPy with mission-specific details.
 

## Full change log

To see a detailed list of all changes, please see the [Full Changelog]().

## Contributors to the v2.0 release

People who have contributed to this release include:
...

In [7]:


### Demo

https://colab.research.google.com/drive/1YA7Jni9ekzmOglg3qRs_tOnBXnFSJtjq?usp=sharing



### Will existing Lightkurve code break?

While we try to maintain a high degree of backwards compatibility with Lightkurve v1.x, it looks like nine API changes are unavoidable:
1. **lc.time is now always an Astropy Time object.**  Unfortunately, `Time` objects cannot be passed to many numpy/scipy functions, you typically have to pass `lc.time.value` instead.
2. **All other columns are now Astropy Quantity objects.** Most numpy/scipy functions can deal with these, but you will nevertheless have to use `lc.flux.value` at times.
3. **All metadata is now stored in the lc.meta dictionary**, including all FITS header values.  For convenience, we support attribute access, e.g. `lc.sector` resolves to `lc.meta['sector']`, but the use of the latter is encouraged because not every light curve will have a sector keyword.
4. **LightCurveFile classes have been removed.** Instead, files must now be loaded using `lightkurve.read(filename, flux_column="pdcsap_flux", quality_bitmask="default")`.
5. **lc.time can no longer contain NaN values**.  This is not allowed by `Time` objects. As a result, `lk.read()` will always ignore rows with `time=NaN`.  Other columns can still have NaNs however.
6. **lc.bin() now takes time_bin_size and time_bin_start arguments.** This is for consistency with `astropy.timeseries.aggregate_downsample`. This significantly alters the behavior of binning, because bins are now defined in time instead of by number of data points.
7. **lc.fold() now returns un-normalized phase values by default.**  This is for consistency with `TimeSeries.fold()`. For the same reason, the `t0` argument has been deprecated in favor of `epoch_time`, and the method now accepts the extra `epoch_phase`, `wrap_phase`, and `normalize_phase` arguments.
8. **lc.append() no longer supports inplace=True.**  This allows us to implement append via a simple call to `astropy.table.vstack`.
9. **Several attributes and methods have been deprecated.**  They still work but will yield a warning and may be removed in the future:
  * `lc.astropy_time` => `lc.time`
  * `lc.time_format` => `lc.time.format`
  * `lc.time_scale` => `lc.time.scale`
  * `lc.flux_quantity` => `lc.flux`
  * `lc.flux_unit` => `lc.flux.unit`
  * `lc.to_table()` => `lc`
  * `lc.to_timeseries()` => `lc`
  * `search_lightcurvefile()` => `search_lightcurve()`

## How do I install Lightkurve v2.0?
Alternatively, you can install this branch using pip as follows:
```
pip install --quiet --upgrade git+https://github.com/barentsen/lightkurve.git@timeseries-refactor
```

SyntaxError: invalid syntax (<ipython-input-7-22813cdfddc2>, line 3)